In [1]:
from matplotlib import pyplot as plt
import matplotlib
import numpy as np
from tqdm import tqdm
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import sys
sys.path.append("/Users/s2569857/Codes/DeepLearningTheory/core")
import empirical_stat as stat

In [2]:
np.random.seed(0)

# Configuration variables

In [3]:
# Number of networks to initialize with random parameters
number_of_networks = 100

# input xgrid for which to produce the corresponding outputs
# reshape is needed for model input
input_xgrid = np.linspace(0.0, 1.0, num=150).reshape(-1,1)

# architecture
# Starting from the first deep layer (input dim = 1)
architecture = [25,20,8]

# activation function
act_func = "tanh"

# output function
output_func = "linear"

# Seed for the initialisation
seed = 1244235632

In [4]:
def generate_sequential_model(outputs=1, 
                   input_layer=None, 
                   nlayers=2, 
                   units=[100,100],
                   train=False,
                   seed = 12345):
  """
  Create a tensorflow sequential model where all intermediate layers have the same size
  This function accepts an already constructed layer as the input.

  All hidden layers will have the same number of nodes for simplicity

  Arguments:
      outputs: int (default=1)
          number of output nodes (how many flavours are we training)
      input_layer: KerasTensor (default=None)
          if given, sets the input layer of the sequential model
      nlayers: int
          number of hidden layers of the network
      units: int
          number of nodes of every hidden layer in the network
      activation: str
          activation function to be used by the hidden layers (ex: 'tanh', 'sigmoid', 'linear')
  """
  if len(units) != nlayers:
      raise Exception("The length of units must match the number of layers.")
  
  model = tf.keras.models.Sequential(name="pdf")
  if input_layer is not None:
      model.add(input_layer)
  for layer in range(nlayers):
      model.add(tf.keras.layers.Dense(units[layer], 
                                      activation=act_func,
                                      kernel_initializer=tf.keras.initializers.GlorotNormal(seed=seed - layer),
                                      #tf.keras.initializers.HeNormal(seed = seed - nlayers)
                                      #bias_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=1, seed=seed - layer)
                                      ),
      )
  model.add(tf.keras.layers.Dense(outputs, 
                                  activation=output_func, 
                                  kernel_initializer=tf.keras.initializers.GlorotNormal(seed=seed - nlayers)
                                  #tf.keras.initializers.HeNormal(seed = seed - nlayers),
                                  #bias_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=1, seed=seed - nlayers)
                                  ))

  if train:
    opt = tf.keras.optimizers.Nadam()
    model.compile(opt, loss="mse")

  return model

In [5]:
nn_models = []
for _ in range(number_of_networks):
  nn_models.append(
    generate_sequential_model(
                    outputs=architecture[-1], 
                    nlayers=len(architecture)-1, 
                    units=architecture[:-1],
                    seed = seed + _),
  )

model_outputs = np.array(
    [nn_models[rep](tf.convert_to_tensor(input_xgrid)).numpy() for rep in range(len(nn_models))]
    )

In [6]:
for out in range(architecture[-1]):
  try:
    k4
  except NameError:
    m2 = stat.compute_m2(model_outputs)[:,out,:,out]
    m4 = stat.compute_m4(model_outputs, out)
    k4 = stat.compute_k4(m2, m4, input_xgrid.size)
    print(f"Output number {out}")
    print("_____________________")
    print("The maximum value of k4 is: ", k4.max())
    print("The minimum value of k4 is: ", k4.min())
  else:
    del k4
    m2 = stat.compute_m2(model_outputs)[:,out,:,out]
    m4 = stat.compute_m4(model_outputs, out)
    k4 = stat.compute_k4(m2, m4, input_xgrid.size)
    print(f"Output number {out}")
    print("_____________________")
    print("The maximum value of k4 is: ", k4.max())
    print("The minimum value of k4 is: ", k4.min())
  del m2, m4

Output number 0
_____________________
The maximum value of k4 is:  0.0
The minimum value of k4 is:  -0.0002984600054488952
Output number 1
_____________________
The maximum value of k4 is:  0.005481258903872691
The minimum value of k4 is:  0.0
Output number 2
_____________________
The maximum value of k4 is:  0.005039175844287459
The minimum value of k4 is:  0.0
Output number 3
_____________________
The maximum value of k4 is:  0.004126040261288299
The minimum value of k4 is:  0.0
Output number 4
_____________________
The maximum value of k4 is:  0.002657879796518759
The minimum value of k4 is:  0.0
Output number 5
_____________________
The maximum value of k4 is:  0.0
The minimum value of k4 is:  -0.00597855125799843
Output number 6
_____________________
The maximum value of k4 is:  0.0
The minimum value of k4 is:  -0.0032179511505071104
Output number 7
_____________________
The maximum value of k4 is:  0.0
The minimum value of k4 is:  -0.002809243915659277


Output number 0
_____________________
The maximum value of k4 is:  0.0005958729860697926
The minimum value of k4 is:  0.0

In [7]:
6.63434157809669e-05 - 6.63434157809669e-05

0.0